# Détection automatique d'erreurs d'OCR

## Installation

In [1]:
!pip install thefuzz[speedup]

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


## Imports

In [2]:
import nltk
from thefuzz import fuzz
from thefuzz import process
from Levenshtein import distance

Nous allons d'abord utiliser la distance de Levenshtein, selon sa formule originale (1965)

In [3]:
distance("bulletin", "ballotins")

3

Nous allons ensuite faire appel à la librairie TheFuzz (anciennement fuzzywuzzy) qui repose sur python-Levenshtein (https://github.com/seatgeek/thefuzz).
Celle-ci permet de normaliser le résultat en calculant un ratio (pourcentage) de similarité entre deux chaînes de caractères :

In [4]:
fuzz.ratio("bulletin", "ballotins")

71

La fonction  `partial_ratio` se focalise sur la sous-chaîne (substring) la plus similaire : 

In [5]:
fuzz.partial_ratio("Grand", "Grand-Place")

100

La fonction `token_sort_ratio` ignore l'ordre des mots :

In [6]:
fuzz.token_sort_ratio("archives de la ville de Bruxelles", "de Bruxelles la ville de archives")

100

La fonction `token_set_ratio` ignore les répétitions de mots :

In [7]:
fuzz.token_set_ratio("plus jamais ça !", "plus jamais Jamais JAMAIS ça !!!")

100

Chargeons à présent le fichier `all.txt` (tronqué) : 

In [8]:
data_path = "../data/all.txt"
LIMIT = 50000000
text = open(data_path).read()[:LIMIT]

tokens = nltk.wordpunct_tokenize(text)
print(f"{len(tokens)} words (tokens) found")

vocab = set(tokens)
print(f"{len(vocab)} different word forms")

12683666 words (tokens) found
159024 different word forms


Nous pouvons identifier les variantes d'un mot par ordre décroissant de similarité : 

In [9]:
word = "exposition"
process.extract(word, vocab, limit=15)

[('EXPOSITION', 100),
 ('exposition', 100),
 ('Exposition', 100),
 ('Expositions', 95),
 ('EXPOSITIONS', 95),
 ('expositions', 95),
 ('exé', 90),
 ('èsi', 90),
 ('îion', 90),
 ('sité', 90),
 ('Itî', 90),
 ('ition', 90),
 ('TION', 90),
 ('tio', 90),
 ('Sîi', 90)]

Les caractères accentués ne sont pas pris en compte pour le calcul du score, mais cela reste une bonne base pour récupérer les occurrences mal OCRisées d'un mot.